In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from functools import partial

import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset

from fastai.basic_train import Learner
from fastai import train
from fastai.basic_data import DataBunch
from fastai.callbacks import EarlyStoppingCallback

from dataloader.toy import ToySinData, ToyQubicData
from model.dense import Dense
from uncertainty_estimator.masks import build_masks, DEFAULT_MASKS
from experiment_setup import build_estimator
from experiments.utils.fastai import ImageArrayDS

# Config
config = {
    'use_cache': False,
    'layers': [1, 128, 128, 64, 1],
    'batch_size': 2,
    'retrain': True,
    'nn_runs': 100,
    'dropout_uq': 0.5,
    'patience': 50
}



In [ ]:
plt.rcParams['figure.facecolor'] = 'white'

In [ ]:
# Import data

data_class = ToySinData
# data_class = ToyQubicData

x_train, y_train = data_class(use_cache=config['use_cache']).dataset('train')
x_val, y_val = data_class(use_cache=config['use_cache']).dataset('val')
x_true, y_true = data_class().dataset('ground_truth')

plt.plot(x_true, y_true)
plt.scatter(x_train, y_train, color='red')
plt.scatter(x_val, y_val, color='green')

train_ds = TensorDataset(torch.FloatTensor(x_train), torch.FloatTensor(y_train))
val_ds = TensorDataset(torch.FloatTensor(x_val), torch.FloatTensor(y_val))
data = DataBunch.create(train_ds, val_ds, bs=config['batch_size'])


In [ ]:
# Train model
model = Dense(config['layers']).float()
loss_func = torch.nn.MSELoss()
callback_fns=[partial(EarlyStoppingCallback, monitor='valid_loss', min_delta=1e-3, patience=config['patience'])]
learner = Learner(data, model, loss_func=loss_func, callback_fns=callback_fns)

model_path = "data/model_masks.pt"
if config['retrain'] or not os.path.exists(model_path):
    learner.fit_one_cycle(1000, 5e-3, wd=1e-3, div_factor=10)
    torch.save(model.state_dict(), model_path)
else:
    model.load_state_dict(torch.load(model_path))


In [ ]:

model.eval()

x_ = np.concatenate((x_true, x_train))
x_tensor = torch.FloatTensor(x_).to('cuda')
x_true_tensor = torch.FloatTensor(x_true).to('cuda')
y_ = model(x_tensor).detach().cpu().numpy()
plt.figure(figsize=(22, 12))
plt.plot(x_true, y_true, alpha=0.5)
plt.scatter(x_train, y_train, color='red')
plt.scatter(x_, y_, color='green', marker='+')


In [ ]:
masks = build_masks(DEFAULT_MASKS)
y_pred = model(x_true_tensor).detach().cpu().numpy()

def make_uq_graph(name, estimations):
    plt.title(name)
    plt.plot(x_true, y_true, alpha=0.5)
    plt.scatter(x_true, y_pred, color='green', marker='+')
    plt.fill_between(np.ravel(x_true), np.ravel(y_pred) - estimations, np.ravel(y_pred) + estimations, alpha=0.3,
                     color='green')
    plt.scatter(x_train, y_train, color='red')


dropout_runs = {}

plt.figure(figsize=(16, 30))
for i, (name, mask) in enumerate(masks.items()):
    if hasattr(mask, 'reset'):
        mask.reset()
    estimator = build_estimator(
        'mcdue_masked', model, keep_runs=True, nn_runs=config['nn_runs'],
        dropout_mask=mask, dropout_rate=config['dropout_uq'])

    estimations = estimator.estimate(x_true_tensor)
    dropout_runs[name] = estimator.last_mcd_runs()
    plt.subplot(6, 2, i + 1)
    make_uq_graph(name, estimations)


In [ ]:
### Dropout predictions
def plot_dropout_runs(name, predictions, n=50):
    plt.figure(figsize=(20, 10))
    plt.title(name)
    plt.plot(x_true, y_true)
    plt.scatter(x_train, y_train, color='red')
    axes = plt.gca()
    axes.set_ylim([-4, 4])
    for prediction_line in predictions[:n]:
        plt.plot(x_true, prediction_line, alpha=0.5)
    plt.show()


for name, runs in dropout_runs.items():
    plot_dropout_runs(name, runs.T, 200)



In [ ]:
### Print sampling
dpp = masks[list(masks.keys())[-1]]
samples = dpp.dpps[0].list_of_samples
lengths = [len(sample) for sample in samples]
print(sum(lengths) / len(lengths) / config['layers'][1])
samples

